In [0]:
# --- IMPORTS (REFATORED) ---
%load_ext autoreload
%autoreload 2

import sys
import os
# Adiciona o diretório atual ao path para encontrar o módulo 'src'
sys.path.append(os.getcwd())

from src.ingestion.connectors import connect_jdbc, url_jdbc
from src.ingestion.feature_store import salvar_feature_table

from pyspark.sql.functions import col, to_date, lit
from databricks.feature_engineering import FeatureEngineeringClient


In [0]:
# --- CONFIGURAÇÕES GLOBAIS DE OTIMIZAÇÃO (BEST PRACTICES) ---
# Ativa otimização automática de gravações e compactação
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")


In [0]:
list_tables = [
'CVC.BIP_vHISTORICO_SUPORTE_CANAL_LOJA_DATABRICKS'
,'CVC.BIP_vHISTORICO_TARGUET_LOJA_DATABRICKS'
,'CVC.BIP_vHISTORICO_FERIADOS_LOJA_DATABRICKS'
,'CVC.BIP_vHISTORICO_SUPORTE_CANAL_ECOMMERCE_DATABRICKS'
,'CVC.BIP_vHISTORICO_TARGUET_ECOMMERCE_DATABRICKS'
,'CVC.BIP_vHISTORICO_SUPORTE_CANAL_VENDA_DIRETA_DATABRICKS'
,'CVC.BIP_vHISTORICO_TARGUET_VENDA_DIRETA_DATABRICKS'
,'CVC.CMC_aLOJAS']

In [0]:
# --- 1. CONFIGURAÇÃO DE CREDENCIAIS E CONEXÃO ---
# Tenta pegar do Key Vault (Databricks Secrets) ou usa valores manuais (placeholder)
try:
    tenant_id = dbutils.secrets.get(scope="ss-keyvault-dev", key="FBC-API-TENANT-ID")
    client_id = dbutils.secrets.get(scope="ss-keyvault-dev", key="FBC-API-CLIENT-ID")
    client_secret = dbutils.secrets.get(scope="ss-keyvault-dev", key="FBC-API-CLIENT-SECRET")
    print("✅ Credenciais recuperadas do Key Vault com sucesso.")
except:
    print("⚠️ Segredos não encontrados. Usando valores manuais.")

scope = "https://database.windows.net/.default"
sql_endpoint = "sql-mi-cvc-prod.database.windows.net" # Ajuste conforme necessário
database_name = "DSP"

# Gera propriedades de conexão
connection_properties = connect_jdbc(tenant_id, client_id, client_secret, scope)
jdbc_url_str = url_jdbc(sql_endpoint, database_name)

# --- 2. MAPEAMENTO DE CHAVES (PKs) E TIMESTAMP ---
# Define PKs e Timestamp para cada tabela para garantir unicidade no Feature Store
table_config = {
    'CVC.BIP_vHISTORICO_SUPORTE_CANAL_LOJA_DATABRICKS': {'pks': ["METRICAS"], 'ts': "DATA"},
    'CVC.BIP_vHISTORICO_TARGUET_LOJA_DATABRICKS': {'pks': ["CODIGO_LOJA"], 'ts': "DATA"},
    'CVC.BIP_vHISTORICO_FERIADOS_LOJA_DATABRICKS': {'pks': ["CODIGO_LOJA"], 'ts': "DATA"},
    'CVC.BIP_vHISTORICO_SUPORTE_CANAL_ECOMMERCE_DATABRICKS': {'pks': ["METRICAS"], 'ts': "DATA"},
    'CVC.BIP_vHISTORICO_TARGUET_ECOMMERCE_DATABRICKS': {'pks': ["CODIGO_LOJA"], 'ts': "DATA"},
    'CVC.BIP_vHISTORICO_SUPORTE_CANAL_VENDA_DIRETA_DATABRICKS': {'pks': ["METRICAS"], 'ts': "DATA"},
    'CVC.BIP_vHISTORICO_TARGUET_VENDA_DIRETA_DATABRICKS': {'pks': ["CODIGO_LOJA"], 'ts': "DATA"},
    'CVC.CMC_aLOJAS': {'pks': ["CODIGO_LOJA"], 'ts': None}
}
# --- 3. LOOP DE INGESTÃO ---
print(f"🚀 Iniciando ingestão de {len(list_tables)} tabelas...")

for table_source in list_tables:
    table_name_clean = table_source.split('.')[-1] # Ex: BIP_vHISTORICO...
    target_table_name = f"cvc.features.{table_name_clean.lower()}" # Ex: cvc.features.bip_vhistorico...
    
    # Verifica configuração
    if table_source in table_config:
        config = table_config[table_source]
        pks = config['pks']
        timestamp_col = config['ts']
    else:
        print(f"⚠️ Configuração não encontrada para {table_source}. Pulando...")
        continue

    print(f"\n📦 Processando: {table_source} -> {target_table_name}")
    
    try:
        # Leitura JDBC
        print(f"   📥 Lendo do SQL Server...")
        df_source = spark.read.jdbc(
            url=jdbc_url_str,
            table=table_source,
            properties=connection_properties
        )
        
        # Opcional: Casting de colunas de data se necessário (Exemplo genérico)
        # if timestamp_col:
        #     df_source = df_source.withColumn(timestamp_col, to_date(col(timestamp_col)))

        # Gravação no Feature Store
        print(f"   💾 Salvando no Feature Store...")
        salvar_feature_table(
            df=df_source,
            table_name_full=target_table_name,
            pk_columns=pks,
            timestamp_col=timestamp_col,
            spark=spark
        )
        print(f"✅ Sucesso: {target_table_name}")
        
    except Exception as e:
        print(f"❌ Erro ao processar {table_source}: {str(e)}")

# Remove variáveis sensíveis da memória
del connection_properties
del client_secret
